In [1]:
import pandas as pd 

In [2]:
Anos_de_vida_saludable_a_los_65_anos = pd.read_excel('data\Anos_de_vida_saludable_a_los_65_anos.xls')

In [3]:
# FUNCION DE LIMPIEZA GENERAL:

def limpieza_columnas(df):
    
    df_copy = df.copy()
    
    years = df_copy.iloc[1].values
    genders = df_copy.iloc[2].values
    
    new_columns = []
    current_year = None
    
    #SEPARACION AÑOS Y GÉNERO
    
    for i in range(len(years)):
        if pd.isna(years[i]):
            new_columns.append(f'{current_year}_{genders[i]}')
        else:
            current_year = years[i]
            new_columns.append(f'{current_year}_{genders[i]}')
    
    df_copy.columns = new_columns
    
    df_copy = df_copy.iloc[3:]
    
    df_copy.set_index(df_copy.columns[0], inplace=True)
    
    # Eliminar las tres últimas filas si tienen más de un 90% de valores nulos, hay algunos excel que tienen nulos al final como observaciones
    
    rows_to_drop = df_copy.tail(3).index[df_copy.tail(3).isnull().mean(axis=1) > 0.9]
    df_copy = df_copy.drop(rows_to_drop)
    
    # QUITAR MULTIINDEX
    
    df_copy = df_copy.reset_index()
      
    df_copy.rename(columns={'None_nan': 'Region'}, inplace=True)
      
    return df_copy

test = limpieza_columnas(Anos_de_vida_saludable_a_los_65_anos)

In [4]:
test

,Region,2017_Hombres,2017_Mujeres,2017_Total,2018_Hombres,2018_Mujeres,2018_Total,2019_Hombres,2019_Mujeres,2019_Total,2020_Hombres,2020_Mujeres,2020_Total,2021_Hombres,2021_Mujeres,2021_Total
0,España (ES),17.3,19.82,18.62,17.31,19.87,18.65,17.83,20.23,19.09,16.64,19.09,17.92,17.36,19.92,18.7
1,Andalucía (AN),15.67,16.66,16.16,15.55,16.64,16.09,17.42,18.81,18.17,16.84,18.24,17.59,16.83,18.24,17.58
2,Aragón (AR),18.66,21.65,20.18,18.73,21.69,20.24,18.08,20.31,19.44,16.79,18.56,17.91,17.56,19.79,18.91
3,"Asturias, Principado de (AS)",16.99,17.67,17.28,16.91,17.76,17.29,16.84,20.1,18.58,15.98,19.32,17.74,16.81,19.77,18.4
4,"Balears, Illes (IB)",18.38,21.74,19.97,18.55,22.05,20.2,19.47,19.58,19.42,18.99,19.38,19.09,18.96,19.37,19.07
5,Canarias (CN),16.68,19.93,18.53,16.44,19.63,18.24,18.68,19.24,18.86,18.61,19.12,18.76,18.63,19.17,18.79
6,Cantabria (CB),18.44,20.11,18.91,18.17,20.01,18.72,16.88,19.55,18.3,16.49,18.9,17.77,16.95,19.64,18.37
7,Castilla y León (CL),17.86,22.71,20.29,17.95,22.53,20.27,18.14,22.1,20.12,16.46,20.18,18.28,17.81,22.09,19.92
8,Castilla - La Mancha (CM),17.51,20.42,18.98,17.9,20.7,19.32,18.73,21.72,20.27,16.25,19.47,17.86,17.97,21.52,19.77
9,Cataluña (CT),16.94,19.49,18.27,16.99,19.52,18.3,18.32,21.97,20.27,16.66,20.2,18.53,17.75,21.62,19.8


In [5]:
# CREACION DE DF PARA COLOCAR LAS REGIONES AÑOS Y GÉNEROS:

def nuevos_df(df):
    
    df_new = pd.DataFrame(columns=['Region', 'Año', 'Hombre', 'Mujer', 'Total'])
    
    years = set(col.split('_')[0] for col in df.columns if '_' in col)
    
    for year in years:
        if f'{year}_Hombres' in df.columns and f'{year}_Mujeres' in df.columns and f'{year}_Total' in df.columns:
            df_year = df[['Region', f'{year}_Hombres', f'{year}_Mujeres', f'{year}_Total']].copy()
            df_year.columns = ['Region', 'Hombre', 'Mujer', 'Total']
            df_year['Año'] = year
            df_new = pd.concat([df_new, df_year], ignore_index=True)
    return df_new

prueba = nuevos_df(test)

prueba

,Region,Año,Hombre,Mujer,Total
0,España (ES),2019,17.83,20.23,19.09
1,Andalucía (AN),2019,17.42,18.81,18.17
2,Aragón (AR),2019,18.08,20.31,19.44
3,"Asturias, Principado de (AS)",2019,16.84,20.1,18.58
4,"Balears, Illes (IB)",2019,19.47,19.58,19.42
...,...,...,...,...,...
95,"Navarra, C. Foral de (NC)",2021,17.26,21.79,19.57
96,País Vasco (PV),2021,17.96,20.28,19.15
97,La Rioja (RI),2021,18.16,19.74,19.1
98,Ceuta (CE),2021,13.68,12.85,13.16
